<a href="https://colab.research.google.com/github/beyg1/Q3/blob/main/Rag_Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
pinecone_api = userdata.get('pinecone_api')
pc = Pinecone(api_key=pinecone_api)

In [3]:
                     # Initialization Pinecone Index
import time
index_name = "test-index"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,      #dimension for google's embeddings model
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [4]:
                  # Initializing Embeddigs
%pip install --upgrade --quiet  langchain-google-genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]
                  # Initializing Vector Store
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [5]:
                     # Passing Data to Pinecone
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['74b940c2-d212-41ad-b443-1461cf8a83fc',
 '6c955c00-f49c-4d78-9837-15e7b57d6151',
 '3860d1e9-6d77-4b79-87f5-db76eb332af0',
 'fafd6069-1c79-45c2-9b44-38a4652c5459',
 'e2f1821f-a915-49ad-85c4-5e894960dd05',
 '396214a1-b722-4537-afe3-1219a8fe7f6f',
 '0b949e7b-93a4-48e9-b744-70bbadb2d133',
 '098b1498-a4fe-4ba0-a4a4-855f30b9a11e',
 'a5657bc9-bdf3-4469-8aa6-1b6ad01a5eab',
 '0ac7ae6a-18c9-4d27-ad89-b1b2fb1390ef']

In [24]:
                       # Data Retrieval
                       #Direct Querry
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [7]:
# Similarity Search
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
    )
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [8]:
# Retrieval Querry by transforming Vec Store into retriever
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

[Document(id='d6223f96-8881-49e5-9e8c-77ce930d4466', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]

Up until now retrieval has been happening after data is embedded into vectors.
Now we will pass the relevant data that has been retrieved into the llm to generate content on the base of retrieved context.

In [32]:
# Selecting Gemini's LLM model
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [33]:
#Making a function that will take user's querry and retrieve similar content
#from vector db and pass it to Gem LLM so that it could generate content based
#on specific context
def llm_answer (querry:str):
  #Vector Search to search for the relevant context based on user querry
  results = vector_store.similarity_search(querry, k=1,)
  #Transfering relevant vectors & user's querry to LLM
  response_llm = llm.invoke(f"Answer user's querry: {querry}, based on the context: {results}")
  return response_llm

In [34]:
response = llm_answer("Analyse the security situation and weather as i am planning a trip.")

In [35]:
response.content

"Okay, based on the information provided, here's an analysis for your trip planning:\n\n**Security Situation:**\n\n*   **No information:** The provided document only contains weather information. There is **no mention of the security situation** in the context. Therefore, I cannot provide any analysis on security. You will need to consult other sources for that information, such as travel advisories, local news, or government websites.\n\n**Weather:**\n\n*   **Cloudy and Overcast:** The weather forecast for tomorrow is described as cloudy and overcast.\n*   **Temperature:** The high temperature is expected to be 62 degrees.\n\n**Summary and Recommendations:**\n\n*   **Weather:** Be prepared for a day with cloud cover. The temperature of 62 degrees is relatively mild, so you might need a light jacket or sweater depending on your comfort level.\n*   **Security:** **You must find information about the security situation from other sources.** Do not rely solely on this information for your